# Building a Simple Agent using LangChain and OpenAI

The purpose of this notebook is to show you how to build a simple agent using [LangChain](https://www.langchain.com/), and [OpenAI](https://openai.com/). The code is based on the ["Build an Agent"](https://python.langchain.com/v0.2/docs/tutorials/agents/) LangChain tutorial, but with modifications to hit a custom API as part of the agent's tools.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from os import environ

if not environ.get("OPENAI_API_KEY"):
    from getpass import getpass
    environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY")

In [3]:
from dataclasses import dataclass
from functools import partial
from random import choices
from string import ascii_lowercase
from typing import Literal

import gradio as gr
import pandas as pd

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.runnables import Runnable
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent


In [4]:
# Initialize the OpenAI chat model (replace "gpt-4o" with your desired model)
model = ChatOpenAI(model="gpt-4o")

In [5]:
# Initialize the checkpoint saver
checkpointer = SqliteSaver.from_conn_string(":memory:")

In [6]:
@dataclass
class Vehicle:
    brand: str
    model: str
    type_: Literal["truck"] | Literal["suv"] | Literal["sedan"] | Literal["hatchback"] | Literal["compact"]
    is_electric: bool
    km_per_liter: int | None
    range_in_km: int | None
    seats: int
    price: int

@tool
def lookup_vehicles(
    brand: str | None = None,
    model: str | None = None,
    type_: str | None = None,
    is_electric: bool | None = None
) -> list[Vehicle]:
    """List vehicles based on their brand, model, type and whether it is electric"""
    df = pd.read_csv("vehicles.csv")

    if brand is not None:
        df = df[df.brand == brand]

    if model is not None:
        df = df[df.model == model]

    if type_ is not None:
        df = df[df.type_ == type_]

    if is_electric is not None:
        df = df[df.is_electric == is_electric]

    return [Vehicle(**record) for record in df.to_dict("records")]

In [7]:
# Initialize the tools
tools = [lookup_vehicles]

In [8]:
# Define the chat prompt template
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(
        content="You are a virtual assistant named 'SimpleAgent'. You are an expert on cars."
    ),
    MessagesPlaceholder("messages")
])

In [9]:
# Create the agent
agent_executor = create_react_agent(
    model,
    tools,
    checkpointer=checkpointer,
    messages_modifier=prompt
)

In [10]:
# Create the Gradio interface
def chat_interface_fn(
    agent: Runnable,
    message: str,
    history: list[list[str]],  # We don't use the ChatInterface history
    thread_id: str
):
    """
    ChatInterface fn argument. This is invoked everytime the
    "Submit" button is pressed on the Gradio interface.
    """
    response = agent.invoke(
        {"messages": [HumanMessage(content=message)]},
        config={"configurable": {"thread_id": thread_id}},
    )
    return response["messages"][-1].content


gr.ChatInterface(
    fn=partial(chat_interface_fn, agent_executor),
    additional_inputs=[
        gr.Textbox("".join(choices(ascii_lowercase, k=8)), label="Thread ID"),
    ],
).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
